In [ ]:
import pandas as pd
from scipy.signal import find_peaks
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [ ]:
kalib = pd.DataFrame()

In [ ]:
def gaussian(x, a, x_0, sigma):
    return a*np.exp(-(x-x_0)**2/sigma**2)

In [ ]:
# Datenimport
kalib['cs'] = np.genfromtxt('data/kalib_cs137.txt')
kalib['cs'] = kalib['cs'].iloc[2:]

kalib['ba'] = np.genfromtxt('data/kalib_ba133.txt')
kalib['ba'] = kalib['ba'].iloc[2:]

kalib['am'] = np.genfromtxt('data/kalib_am241.txt')
kalib['am'] = kalib['am'].iloc[2:]
kalib.dropna(inplace=True)
kalib['N'] = np.asarray(range(16382))+1

In [ ]:
kalib.drop(columns=['N']).iloc[:1500].plot()
kalib['cs'].iloc[:1500].plot()

In [ ]:
# I. Kalibrierung
linear = []

In [ ]:
kalib['am'].iloc[1050:1090].plot()
am_1_fit, am_1_dev = curve_fit(gaussian, kalib['N'].iloc[1050:1090], kalib['am'].iloc[1050:1090], p0=[10, 1072, 10])
plt.plot([i for i in  range(1050, 1090)],[gaussian(i-1, am_1_fit[0], am_1_fit[1], am_1_fit[2]) for i in range(1050, 1090)])
linear.append([am_1_fit[1], 59.54092, np.sqrt(am_1_dev[1][1])])

In [ ]:
kalib['am'].iloc[2120:2170].plot()
am_2_fit, am_2_dev = curve_fit(gaussian, kalib['N'].iloc[2120:2170], kalib['am'].iloc[2120:2170], p0=[10, 2145, 10])
plt.plot([i for i in  range(2120, 2170)],[gaussian(i-1, am_2_fit[0], am_2_fit[1], am_2_fit[2]) for i in range(2120, 2170)])
linear.append([am_2_fit[1], 102.959, np.sqrt(am_2_dev[1][1])])

In [ ]:
kalib['cs'].iloc[550:600].plot()
cs_1_fit, cs_1_dev = curve_fit(gaussian, kalib['N'].iloc[550:600], kalib['cs'].iloc[550:600], p0=[10, 580, 10])
plt.plot([i for i in  range(550, 600)],[gaussian(i-1, cs_1_fit[0], cs_1_fit[1], cs_1_fit[2]) for i in range(550, 600)])
linear.append([cs_1_fit[1], 32.0604505300353, np.sqrt(cs_1_dev[1][1])])

In [ ]:
kalib['cs'].iloc[630:680].plot()
cs_2_fit, cs_2_dev = curve_fit(gaussian, kalib['N'].iloc[630:680], kalib['cs'].iloc[630:680], p0=[10, 650, 10])
plt.plot([i for i in  range(630, 680)],[gaussian(i-1, cs_2_fit[0], cs_2_fit[1], cs_2_fit[2]) for i in range(630, 680)])
# Nicht verwenden.


In [ ]:
kalib['ba'].iloc[530:580].plot()
ba_1_fit, ba_1_dev = curve_fit(gaussian, kalib['N'].iloc[530:580], kalib['ba'].iloc[530:580], p0=[10, 555, 10])
plt.plot([i for i in  range(530, 580)],[gaussian(i-1, ba_1_fit[0], ba_1_fit[1], ba_1_fit[2]) for i in range(530, 580)])
# 30.8
linear.append([ba_1_fit[1], 30.8, np.sqrt(ba_1_dev[1][1])])

In [ ]:
kalib['ba'].iloc[940:970].plot()
ba_3_fit, ba_3_dev = curve_fit(gaussian, kalib['N'].iloc[940:970], kalib['ba'].iloc[940:970], p0=[10, 960, 10])
plt.plot([i for i in  range(940, 970)],[gaussian(i-1, ba_3_fit[0], ba_3_fit[1], ba_3_fit[2]) for i in range(940, 970)])
# 30.8
linear.append([ba_3_fit[1], 53.1622, np.sqrt(ba_3_dev[1][1])])

In [ ]:
kalib['ba'].iloc[1440:1470].plot()
ba_2_fit, ba_2_dev = curve_fit(gaussian, kalib['N'].iloc[1440:1470], kalib['ba'].iloc[1440:1470], p0=[10, 1458, 10])
plt.plot([i for i in  range(1440, 1470)],[gaussian(i-1, ba_2_fit[0], ba_2_fit[1], ba_2_fit[2]) for i in range(1440, 1470)])
# 80.9979
linear.append([ba_2_fit[1], 80.9979, np.sqrt(ba_2_dev[1][1])])
linear

In [ ]:
def lin_fit(x, m, b, c):
    return m*x**2 + b*x + c
    # return b*x + c
plt.errorbar(x=[linear[i][0] for i in range(6)], y=[linear[i][1] for i in range(6)], xerr=[linear[i][2] for i in range(6)], fmt='x', label='Peaks')
regr_fit, regr_dev = curve_fit(lin_fit, [linear[i][0] for i in range(6)], [linear[i][1] for i in range(6)])
plt.plot([i for i in range(500, 2200)], [lin_fit(i, regr_fit[0], regr_fit[1], regr_fit[2]) for i in range(500, 2200)], label='Fit')
plt.legend()
plt.xlabel('Kanal [-]')
plt.ylabel('Photonenenergie [keV]')
plt.savefig('images/kalibration.png', dpi=300)

In [ ]:
def e_of_k(k):
    return lin_fit(k, regr_fit[0], regr_fit[1], regr_fit[2])

In [ ]:
def delta_e(k):
    return np.sqrt(k**4 * regr_dev[0][0] + k**2 * regr_dev[1][1] + regr_dev[2][2])

In [ ]:
# IV Bestimmung der Energie gestreuter Photonen
# Datenimport
winkel = pd.DataFrame()
winkel['30_Alu'] = np.genfromtxt('data/IV_30_Alu.txt')
winkel['40_Alu'] = np.genfromtxt('data/IV_40_Alu.txt')
winkel['50_Alu'] = np.genfromtxt('data/IV_50_Alu.txt')
winkel['60_Alu'] = np.genfromtxt('data/IV_60_Alu.txt')
winkel['70_Alu'] = np.genfromtxt('data/IV_70_Alu.txt')
winkel['80_Alu'] = np.genfromtxt('data/IV_80_Alu.txt')
winkel['90_Alu'] = np.genfromtxt('data/IV_90_Alu.txt')
winkel['100_Alu'] = np.genfromtxt('data/IV_100_Alu.txt')
winkel['110_Alu'] = np.genfromtxt('data/IV_110_Alu.txt')
winkel['120_Alu'] = np.genfromtxt('data/IV_120_Alu.txt')
winkel['130_Alu'] = np.genfromtxt('data/IV_130_Alu.txt')
winkel['30_N0'] = np.genfromtxt('data/IV_30_N0.txt')
winkel['40_N0'] = np.genfromtxt('data/IV_40_N0.txt')
winkel['50_N0'] = np.genfromtxt('data/IV_50_N0.txt')
winkel['60_N0'] = np.genfromtxt('data/IV_60_N0.txt')
winkel['70_N0'] = np.genfromtxt('data/IV_70_N0.txt')
winkel['80_N0'] = np.genfromtxt('data/IV_80_N0.txt')
winkel['90_N0'] = np.genfromtxt('data/IV_90_N0.txt')
winkel['100_N0'] = np.genfromtxt('data/IV_100_N0.txt')
winkel['110_N0'] = np.genfromtxt('data/IV_110_N0.txt')
winkel['120_N0'] = np.genfromtxt('data/IV_120_N0.txt')
winkel['130_N0'] = np.genfromtxt('data/IV_130_N0.txt')

In [ ]:
# Berechnung der Zählraten
winkel_rate = pd.DataFrame()
for i in winkel.columns:
    winkel_rate[i] = winkel[i].iloc[2:] / (winkel[i].iloc[0]-winkel[i].iloc[1])

# NG - N0
winkel_cleaned = pd.DataFrame()
for i in range(11):
    winkel_cleaned[winkel_rate.columns[i]] = winkel_rate[winkel_rate.columns[i]] - winkel_rate[winkel_rate.columns[i+11]]
winkel_cleaned['N'] = np.asarray(range(16382))+1


In [ ]:
winkel_cleaned.drop(columns='N').iloc[800:1100].plot()
winkel_cleaned.sum()

In [ ]:
from kleinnishina import kleinnishina
from kleinnishina import mu as mu_function

In [ ]:
winkel_bin = []
n_int = []
n_int_err = []
for i in range(11):
    fit, std = curve_fit(gaussian, winkel_cleaned['N'].iloc[800:1100], winkel_cleaned[winkel_cleaned.columns[i]].iloc[800:1100], p0=[1, 950, 10])
    winkel_bin.append(fit[1])
    # n_int.append(fit[0]*np.sqrt(np.pi*fit[2]))
    n_int.append(winkel_cleaned[winkel_cleaned.columns[i]].iloc[800:1100].sum())
    n_int_err.append(np.sqrt(fit[2]*np.pi*std[0][0]+std[2][2]*(fit[0]*np.pi)**2/(4*fit[2]*np.pi)))

mu = np.cos(np.pi * np.linspace(30, 130, 11)/180)
mu_err = abs(np.sin(np.pi * np.linspace(30, 130, 11)/180)*2*np.pi/180)
energien = [lin_fit(i, regr_fit[0], regr_fit[1], regr_fit[2]) for i in winkel_bin]


In [ ]:
# Berechnung Proportionalitätsfaktor wirkungsquerschnitt und Zählrate
lamda = [ kleinnishina(i) for i in np.linspace(30, 130, 11)] / np.asarray(n_int)
lamda_opt = lamda.mean()
lamda_std = lamda.std(ddof=1)
lamda_opt*1e31
lamda_std*1e31/np.sqrt(11)

In [ ]:
plt.errorbar(mu,1e31*lamda_opt*np.asarray(n_int), xerr=mu_err, fmt='x', yerr=np.sqrt(lamda_opt**2*np.asarray(n_int_err)**2 + lamda_std**2*1e31*np.asarray(n_int)), label='Messwerte')
plt.plot([mu_function(i) for i in np.linspace(30, 130, 100)], [ 1e31*kleinnishina(i) for i in np.linspace(30, 130, 100)], label='$\sigma_{\Omega}^{KN}$')
plt.xlabel('µ')
plt.ylabel('$\sigma$ [mb]')
plt.legend()
plt.savefig('images/wirkungsquerschnitt.png', dpi=300)

In [ ]:
delta_x = linear[2][2]
energien_fehler = np.sqrt(np.asarray(energien)**4 * regr_dev[0][0]+ np.asarray(energien)**2 * regr_dev[1][1] + regr_dev[2][2] + (2*regr_fit[0]*np.asarray(energien) + regr_fit[1])**2 * delta_x**2)
plt.errorbar(mu, energien, xerr=mu_err, yerr=energien_fehler, fmt='x')
plt.xlabel('µ [-]')
plt.ylabel('Energie [keV]')
plt.plot([ np.cos( np.pi * i / 180) for i in np.linspace(30, 130, 100) ], [ e_of_theta(i) for i in np.linspace(30, 130, 100)])
plt.savefig('images/winkelenergie.png', dpi=300)


In [ ]:
# Histogramm
N = np.genfromtxt('data/II_histogramm_Am241.txt')
N_0 = np.genfromtxt('data/III_histogramm_Nullmessung.txt')

In [ ]:
N_dot = pd.DataFrame()
N_dot['N'] = N[2:] / (N[0]-N[1])
N_dot['N_0'] = N_0[2:] / (N_0[0]-N_0[1])

In [ ]:
# N_dot.drop(columns=['hist', 'K']).iloc[:2000].plot()
N_dot['hist'] = N_dot['N'] - N_dot['N_0']
N_dot['K'] = np.asarray(range(16382)) + 1
N_dot['E'] = [ e_of_k(i) for i in N_dot['K']]
plt.plot(N_dot['E'].iloc[500:2000], N_dot['N'].iloc[500:2000], label='$\dot{N}_G$')
plt.plot(N_dot['E'].iloc[500:2000], N_dot['N_0'].iloc[500:2000], label='$\dot{N}_0$')
plt.xlabel('Energie [kev]')
plt.legend()
plt.ylabel('Zählrate [s⁻¹]')
plt.savefig('images/p0g.png', dpi=300)
# plt.ylabel('$\dot{N} = \dot{N}_G - \dot{N}_0$')


In [ ]:
plt.plot(N_dot['K'].iloc[800:1250], N_dot['hist'].iloc[800:1250])
hist_fit, hist_dev = curve_fit(gaussian, N_dot['K'].iloc[800:1250], N_dot['hist'].iloc[800:1250], p0=[1, 960, 10])
plt.plot(N_dot['K'].iloc[800:1250], [ gaussian(i, hist_fit[0], hist_fit[1], hist_fit[2]) for i in N_dot['K'].iloc[800:1250]])
hist_fit

hist_1_fit, hist_1_dev = curve_fit(gaussian, N_dot['K'].iloc[800:1250], N_dot['hist'].iloc[800:1250], p0=[1, 1075, 10])
plt.plot(N_dot['K'].iloc[1050:1250], [ gaussian(i, hist_1_fit[0], hist_1_fit[1], hist_1_fit[2]) for i in N_dot['K'].iloc[1050:1250]])


In [ ]:
plt.plot([ e_of_k(i) for i in N_dot['K'].iloc[800:1150]], N_dot['hist'].iloc[800:1150])
hist_fit, hist_dev = curve_fit(gaussian, [ e_of_k(i) for i in N_dot['K'].iloc[800:1250]], N_dot['hist'].iloc[800:1250], p0=[1, 50, 10])
plt.plot([ e_of_k(i) for i in N_dot['K'].iloc[800:1050]], [ gaussian(i, hist_fit[0], hist_fit[1], hist_fit[2]) for i in [ e_of_k(i) for i in N_dot['K'].iloc[800:1050]]])
# plt.plot([ e_of_k(i) for i in N_dot['K'].iloc[1050:1100]], N_dot['hist'].iloc[1050:1100])
hist_1_fit, hist_1_dev = curve_fit(gaussian, [ e_of_k(i) for i in N_dot['K'].iloc[1050:1100]], N_dot['hist'].iloc[1050:1100], p0=[1, 61, 2])
plt.plot([ e_of_k(i) for i in N_dot['K'].iloc[1050:1150]], [ gaussian(i, hist_1_fit[0], hist_1_fit[1], hist_1_fit[2]) for i in [ e_of_k(i) for i in N_dot['K'].iloc[1050:1150]]])
plt.xlabel('Energie [kev]')
plt.ylabel('Zählrate [s⁻¹]')
plt.savefig('images/histfit.png', dpi=300)